In [1]:
import pandas as pd
import numpy as np
import datetime
import os

print(datetime.datetime.now())
print(os.getcwd())

2020-12-11 12:35:17.804380
/mnt/clients/juba/hqjubaapp02/sharefolder/CC


In [2]:
excel_file=pd.ExcelFile("./CC GA TA Analyses.xlsx")
excel_file.sheet_names

['Sheet1', 'Zipcodes']

In [3]:
df_output=excel_file.parse("Sheet1",dtype=str)
df_zip_city=excel_file.parse("Zipcodes",dtype=str)

In [4]:
print(df_zip_city.shape,df_zip_city['zip_cd'].nunique())
df_zip_city=df_zip_city[pd.notnull(df_zip_city['zip_cd'])]
print(df_zip_city['zip_cd'].apply(len).unique())
df_zip_city['zip_cd']=df_zip_city['zip_cd'].str.zfill(5)
print(df_zip_city['zip_cd'].apply(len).unique())

(9317, 4) 9024
[5 4]
[5]


In [5]:
df_EASI_A=pd.read_csv("/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/EASI/2018-07-19/ZIP4_18_DATA_A2_CSV/ZIP4_18_DATA_A2.CSV",
                     skiprows=1,usecols=['ZIP_CODE','ZIP4','POP18'],dtype=str)
df_EASI_A['POP18']=df_EASI_A['POP18'].astype(float)

df_EASI_C=pd.read_csv("/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/EASI/2018-07-19/ZIP4_18_DATA_C2_CSV/ZIP4_18_DATA_C2.CSV",
                     skiprows=1,usecols=['ZIP_CODE','ZIP4','PCTHH55_64','PCTHH65_74','PCTHH75P'],dtype=str)
df_EASI_C['PCTHH55_64']=df_EASI_C['PCTHH55_64'].astype(float)
df_EASI_C['PCTHH65_74']=df_EASI_C['PCTHH65_74'].astype(float)
df_EASI_C['PCTHH75P']=df_EASI_C['PCTHH75P'].astype(float)
df_EASI_C['PCTHH_55P']=df_EASI_C['PCTHH55_64']+df_EASI_C['PCTHH65_74']+df_EASI_C['PCTHH75P']
del df_EASI_C['PCTHH55_64']
del df_EASI_C['PCTHH65_74']
del df_EASI_C['PCTHH75P']
print(df_EASI_A.shape)
print(df_EASI_C.shape)
df_EASI_A=pd.merge(df_EASI_A,df_EASI_C,on=['ZIP_CODE','ZIP4'],how="left")
df_EASI_A['Pop_55_plus']=df_EASI_A['POP18']*df_EASI_A['PCTHH_55P']
df_EASI_A['Pop_55_plus']=df_EASI_A['Pop_55_plus']/100
print(df_EASI_A.shape)

(40306307, 3)
(40306307, 3)
(40306307, 5)


In [6]:
df_EASI_A.head(4)

,ZIP_CODE,ZIP4,POP18,PCTHH_55P,Pop_55_plus
0,00501,0001,0.0,0.00,0.00000
1,00501,0002,0.0,0.00,0.00000
2,01001,0001,114.2,42.81,48.88902
3,01001,0121,102.5,42.81,43.88025


In [7]:
df_pop_55_plus_by_zip=df_EASI_A.groupby("ZIP_CODE")['Pop_55_plus'].sum().to_frame().reset_index()
df_pop_55_plus_by_zip=df_pop_55_plus_by_zip.rename(columns={"ZIP_CODE":"zip_cd"})
print(df_pop_55_plus_by_zip['zip_cd'].apply(len).unique())
print(df_zip_city['zip_cd'].apply(len).unique())


[5]
[5]


In [8]:
df_zip_city=pd.merge(df_zip_city,df_pop_55_plus_by_zip,on="zip_cd",how="left")
df_zip_city.head(2)

,State,City,zip_cd,Dma,Pop_55_plus
0,New York,New York,10169,NEW YORK,0.0
1,New York,New York,10168,NEW YORK,0.0


In [9]:
del df_EASI_A
del df_EASI_C
import gc
gc.collect()

55

In [10]:
df_output_population=df_zip_city.groupby(["State",'City'])['Pop_55_plus'].sum().to_frame().reset_index()
df_output_zip_count=df_zip_city.groupby(["State",'City'])['zip_cd'].nunique().to_frame().reset_index().rename(columns={"zip_cd":"qc_zip_count"})


In [11]:
df_output=pd.merge(df_output,df_output_population,on=["State",'City'],how="left")
df_output=pd.merge(df_output,df_output_zip_count,on=["State",'City'],how="left")
df_output.head(20)

,State,City,#zipcodes,Pop 55+,Pop_55_plus,qc_zip_count
0,New York,New York,115,NaN,627340.43264,115.0
1,Ohio,Cleveland,35,NaN,337220.06201,35.0
2,Illinois,Chicago,78,NaN,983610.81815,78.0
3,Ohio,Columbus,40,NaN,300140.66719,40.0
4,Georgia,Atlanta,83,NaN,329752.19120,83.0
5,Texas,Dallas,88,NaN,468725.13342,88.0
6,California,Los Angeles,90,NaN,889976.74231,90.0
7,Florida,Orlando,55,NaN,326986.24683,55.0
8,District of Columbia,Washington,216,NaN,242767.43440,216.0
9,Tennessee,Nashville,39,NaN,168607.69236,39.0


In [12]:
df_output.to_csv("./CC_city_with_population_JL_%s.csv"%str(datetime.datetime.now().date()),index=False)